In [3]:
import cv2
import numpy as np
import pandas as pd
import joblib
import mediapipe as mp

###### Chargement de modèle entrainé avec joblib

In [4]:
svm_model_loaded = joblib.load('asl_svm_model.pkl')

###### Extraction et représentation des landmarks de la main sur l'image.

In [5]:
def extract_landmarks(image, results, mpDraw):
    # Extraction des dimensions de l'image
    h, w, c = image.shape
    # Initialisation du tableau pour stocker les coordonnées des landmarks
    landmarks_array = []
    
    # Vérification de la présence de landmarks pour une seule main dans les résultats
    if results.multi_hand_landmarks:
        # Récupération des landmarks de la première main détectée
        hand_landmarks = results.multi_hand_landmarks[0]
            
        # Extraction des coordonnées des landmarks
        landmarks = hand_landmarks.landmark
        landmarks_row = []
            
        # Boucle à travers chaque landmark
        for landmark in landmarks:
                
            # Dessin d'un cercle pour représenter chaque landmark sur l'image
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            
             # Stockage des coordonnées des landmarks dans le tableau
            landmarks_row.extend([cx, cy])
            cv2.circle(image, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                
        # Stockage des coordonnées des landmarks de la main dans le tableau principal
        landmarks_array.append(landmarks_row)
        
        # Dessin des connexions entre les landmarks de la main sur l'image
        mpDraw.draw_landmarks(image, hand_landmarks, mpHands.HAND_CONNECTIONS)
        
        # Retour de l'image avec les landmarks dessinés et des coordonnées des landmarks
        return image, np.array(landmarks_array).flatten()
    else:
        # Si aucun landmark n'est détecté ou si plusieurs mains sont détectées, retourne None
        return None


###### Configuration de la capture vidéo de la webcam

In [10]:
# Capture vidéo en direct à partir de la webcam (index 0)
cap = cv2.VideoCapture(0)

# Taille désirée pour la webcam
webcam_width, webcam_height = 400, 400

# # Ajustement de la taille de la webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, webcam_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, webcam_height)

True

In [11]:
import time

# Initialisation du temps écoulé
start_time = time.time()

while True:
    ret, frame = cap.read()  # Lecture d'une image de la webcam
    
    if not ret:
        break  # Sortir de la boucle si la lecture de la webcam échoue
    
    # Calculer le temps écoulé depuis la dernière capture de frame
    elapsed_time = time.time() - start_time
    
    if elapsed_time >= 0.5:  # Capturer un frame toutes les 0.5 seconde
        start_time = time.time()  # Réinitialiser le temps de départ
        
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8))  # Traitement de l'image pour détecter les mains

        if results.multi_hand_landmarks:  # Si des landmarks de main sont détectés
            frame_marked, landmarks = extract_landmarks(frame, results, mpDraw)  # Extraction et dessin des landmarks de la main

            landmarks_dataframe = pd.DataFrame([landmarks])  # Conversion des landmarks en DataFrame
            class_ = svm_model_loaded.predict(landmarks_dataframe)  # Prédiction de la classe du geste de la main

            # Affichage de la classe prédite sur l'image
            cv2.putText(frame_marked, class_[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
            cv2.imshow('Classification en temps réel', frame_marked)  # Affichage de l'image avec les landmarks de la main et la classe prédite

        else:
            cv2.imshow('Classification en temps réel', frame)  # Affichage de l'image de la webcam si aucun landmark de main n'est détecté
    
    # Quitter la boucle si la touche 'q' est enfoncée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libération des ressources
cap.release()
cv2.destroyAllWindows()


###### Classification en temps réel des gestes de la main

In [9]:
mpHands = mp.solutions.hands  # Importation du modèle de détection des mains
hands = mpHands.Hands()  # Initialisation du détecteur de mains
mpDraw = mp.solutions.drawing_utils  # Utilitaire pour dessiner les landmarks des mains

while True:
    ret, frame = cap.read()  # Lecture d'une image de la webcam
    
    if not ret:
        break  # Sortir de la boucle si la lecture de la webcam échoue
        
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8))  # Traitement de l'image pour détecter les mains

    if results.multi_hand_landmarks:  # Si des landmarks de main sont détectés
        frame_marked, landmarks = extract_landmarks(frame, results, mpDraw)  # Extraction et dessin des landmarks de la main
        
#         header =[f'ld_1_{i}' for i in range(1, 64)] + ['o_h'] + [f'ld_2_{i}' for i in range(1, 64)]
        landmarks_dataframe = pd.DataFrame([landmarks])  # Conversion des landmarks en DataFrame
        class_ = svm_model_loaded.predict(landmarks_dataframe)  # Prédiction de la classe du geste de la main
        
        # Affichage de la classe prédite sur l'image
        cv2.putText(frame_marked, class_[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        cv2.imshow('Classification en temps réel', frame_marked)  # Affichage de l'image avec les landmarks de la main et la classe prédite
    
    else:
        cv2.imshow('Classification en temps réel', frame)  # Affichage de l'image de la webcam si aucun landmark de main n'est détecté
    
    # Quitter la boucle si la touche 'q' est enfoncée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libération des ressources
cap.release()
cv2.destroyAllWindows()

In [55]:
# Libérer la capture vidéo et fermer les fenêtres OpenCV
cap.release()
cv2.destroyAllWindows()